In [39]:
import numpy as np
import torch
from bertviz import head_view
from attention_graph_util import compute_joint_attention
from transformers import BertTokenizer, BertModel

In [65]:
def show_raw_head_view(model, tokenizer, sentence_a, sentence_b=None, layer=None, heads=None):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    if sentence_b:
        token_type_ids = inputs['token_type_ids']
        output = model(input_ids, token_type_ids=token_type_ids)
        attention = output['attentions']
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        output = model(input_ids)
        attention = output['attentions']
        sentence_b_start = None
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
    head_view(attention, tokens, sentence_b_start, layer=layer, heads=heads)

In [74]:
def show_rollout_head_view(model, tokenizer, sentence_a, sentence_b=None, layer=None, heads=None):
    def do_rollout(attention):
        attention = np.array([a.detach().numpy() for a in attention])
        attention = np.swapaxes(attention, 0, 1)
        attention = compute_joint_attention(attention, layer_dim=0)
        attention = np.swapaxes(attention, 0, 1)
        attention = [torch.from_numpy(a) for a in attention]
        return attention
    
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    if sentence_b:
        token_type_ids = inputs['token_type_ids']
        output = model(input_ids, token_type_ids=token_type_ids)
        attention = output['attentions']
        attention = do_rollout(attention)
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        output = model(input_ids)
        attention = output['attentions']
        attention = do_rollout(attention)
        sentence_b_start = None
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
    head_view(attention, tokens, sentence_b_start, layer=layer, heads=heads)

In [73]:
model_version = 'bert-base-uncased'
do_lower_case = True
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)
sentence_a = "The author talked to Sara about "+tokenizer.mask_token+" book"
#sentence_a = "the rabbit quickly hopped"
#sentence_b = "The turtle slowly crawled"
show_raw_head_view(model, tokenizer, sentence_a)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<IPython.core.display.Javascript object>

In [75]:
show_rollout_head_view(model, tokenizer, sentence_a)

<IPython.core.display.Javascript object>